In [25]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# Recieving Fact of the Day

In [26]:
import json
from openai import OpenAI

client = OpenAI()


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a knowledge assistant. You can answer questions about any topic."},
        {"role": "user", "content": "Tell me a random fact of the day, based on information you found in the internet. I want to learn something new and it is important that I have a valid source, so be 100% sure to tell only true and reliable facts. Don't tell me the source as link, rather tell me the provider (Organisation, Magazine, Websitename, etc.). Give me the fact and its source in JSON format like this: {\"fact\": \"Here is the fact\", \"source\": \"Here is the source\"}. Be sure that you don't include any other information in the response."},
    ]
)
assistant_reply = json.loads(response.choices[0].message.content)

print(assistant_reply)
fact = assistant_reply['fact']
source = assistant_reply['source']  

{'fact': 'The average person walks the equivalent of five times around the world in their lifetime.', 'source': 'BBC News'}


# Recieving Background-Picture

In [29]:
import urllib.request, datetime, os
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model = "dall-e-3",
  prompt=f"{fact}. Generate a picture which suits to that fact. Dont add any text on the picture and avoid symbols.",
  size="1024x1024",
  quality="standard",
  n=1
)
image_url = response.data[0].url
print(image_url)

now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d-%H%M%S")

MYDIR = ("imageOfTheDay")
CHECK_FOLDER = os.path.isdir(MYDIR)
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("Created folder : ", MYDIR)

picture = urllib.request.urlretrieve(image_url, f"imageOfTheDay/{now}.png")

https://oaidalleapiprodscus.blob.core.windows.net/private/org-wptybHHtLv7LMzdg4i3YhJt4/user-ypp0lyHryGL8poRbrZOKGJCr/img-3hSf4OxTCJKjDlYJxCPhvMeM.png?st=2023-11-10T17%3A44%3A07Z&se=2023-11-10T19%3A44%3A07Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-11-10T14%3A23%3A54Z&ske=2023-11-11T14%3A23%3A54Z&sks=b&skv=2021-08-06&sig=gJRQ7/6WOGf5pc2IbZIsJdDLyYRYP/Ggie5H3%2BN2JMY%3D


# Add content to picture

In [30]:
from PIL import Image, ImageDraw, ImageFont
import textwrap

# Make picture less transparent
image = Image.open(f"imageOfTheDay/{now}.png")
image.putalpha(127)

# Add fact to picture
from PIL import Image, ImageDraw, ImageFont
draw = ImageDraw.Draw(image)
font_path = "/System/Library/Fonts/Helvetica.ttc"
font = ImageFont.truetype(font_path, 36)
text_color = (255, 255, 255) 
text_position = (image.width // 2, image.height // 2) 
max_text_width = 0.8 * image.width 
wrapper = textwrap.TextWrapper(width=30) 
text = wrapper.fill(fact)
draw.text(text_position, text, fill=text_color, font=font, anchor="mm",align="center")

#Add source to picture
additional_font_size = 24 
additional_font = ImageFont.truetype(font_path, additional_font_size)
additional_text_color = (128, 128, 128) 
additional_text_position = (20, image.height - 40) 
draw.text(additional_text_position, f"Source: {source}", fill=additional_text_color, font=additional_font)

# Save picture
image.save(f"imageOfTheDay/{now}.png")
